In [1]:
end_of_string = '\u1026' # character at the end of each string

### Bartosz Kaszuba

# Trie

In [2]:
class Trie:
    class Trie_Node:
        def __init__(self, letter, parent, children = None, link = None):
            self.letter = letter
            self.parent = parent
            
            if link is None:
                self.link = self
            else:
                self.link = link
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def is_child(self,letter):
            return letter in self.children.keys()
        
        def get_child(self, letter):
            return self.children[letter]
        
            
    def __init__(self, text):
        
        self.root = self.Trie_Node(None, None)
        
        text = self.check_text(text)
        
        longest = self.Trie_Node(text[0], self.root, link=self.root)
        self.root.add_child(longest)
        
        for i in range(1,len(text)):
            longest = self.__expand_suffixes(longest, text, i)
            
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def __expand_suffixes(self, longest ,text, cur_pos):
        
        current = longest
        previous = None
        
        while not current.is_child(text[cur_pos]):
            
            child = self.Trie_Node(text[cur_pos], current)
            current.add_child(child)
            # print(current.children)
            # print(current is self.root)

            
            if previous is not None:
                previous.link = child
                
            current = current.link
            previous = child
            
        if current is self.root and self.root.get_child(text[cur_pos]) is previous:
            previous.link = self.root
        else:
            previous.link = current.get_child(text[cur_pos])
            
        return longest.get_child(text[cur_pos])
        
        
            
    def is_in_text(self, text):
        
        pos = 0
        node = self.root

        while node.is_child(text[pos]):

            node = node.get_child(text[pos])
            pos += 1
            if pos == len(text):
                return True
 
        return False
    
    def print_Trie(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
            


Alternatywny sposób konstrukcji drzewa Trie z linkami. Stworzenie sufiksu najmniejszego prefiksu a następnie tworzenie sufiksów kolejnych prefiksów poprzez wydłużanie istniejących i dodanie ostatniej litery każdego prefiksu

# Link Trie

In [3]:
class Link_Trie:
    class Trie_Node:
        def __init__(self, letter, parent, children = None, link = None, depth = 0):
            self.letter = letter
            self.parent = parent
            self.depth = depth
            
            self.link = link
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def is_child(self,letter):
            return letter in self.children.keys()
        
        def get_child(self, letter):
            return self.children[letter]
        
            
    def __init__(self, text):
        
        self.root = self.Trie_Node(None, None)
        self.root.link = self.root
        
        text = self.check_text(text)
        
        head = self.root
        
        for i in range(0,len(text)):
            
            leaf = self.__add_suffix(text, i, head)
            head = self.__up_link_down(leaf)
                        
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def __add_suffix(self, text ,pos, node):
        
        curr = node
        
        for i in range(pos + node.depth, len(text)):
            
            child = self.Trie_Node(text[i], curr, depth = curr.depth+1)
            curr.add_child(child)
            curr = child
            
        return curr
    
    def __up_link_down(self, leaf):
        
        curr = leaf
        queue = []
        
        
        
        while curr.link is None:
            queue.append(curr.letter)
            curr = curr.parent
            
        node = curr.link
        
        if curr is self.root:
            char = queue.pop()
            curr = curr.get_child(char)
            curr.link = self.root
        
        while (queue):
            char = queue.pop()
            if (node.is_child(char)):
                node = node.get_child(char)
                curr = curr.get_child(char)
                curr.link = node
            else:
                break
        return node
        
        
            
    def is_in_text(self, text):
        
        pos = 0
        node = self.root

        while node.is_child(text[pos]):

            node = node.get_child(text[pos])
            pos += 1
            if pos == len(text):
                return True
            
        return False
    
    def print_Trie(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
            


Drzewo Trie stworzone metodą przedstawioną na wykładzie. Algorytm tworzy najpierw najdłuższy sufiks a następnie tworzy następne aktualizując istniejące linki.

# Suffix Tree

In [4]:
class Suffix_Tree:
    
    class Suffix_Node:
        def __init__(self, text, start, length, parent, children = None, link = None, depth = 0):
            
            self.letter = text
            self.start = start
            self.len = length
            self.parent = parent
            self.depth = depth
            
            self.link = link
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def delete_child(self, child):
            self.children.pop(child.letter, None)
            
            
        def is_child(self,text):
            return text in self.children.keys()
        
        def get_child(self, text):
            return self.children[text]
        
        def get_child_by_first_letter(self, letter):
            for key in self.children.keys():
                if key[0] == letter:
                    return self.children[key]
            return None
        
            
    def __init__(self, text):
        
        self.root = self.Suffix_Node(None, 0, 0, None)
        text = self.check_text(text)
        self.text = text
        
        for i in range(0,len(text)):
            
            head = self.slow_find(self.root, text, i)
            
            if head is self.root:
                child = self.Suffix_Node(text[i] ,i ,
                                len(text) - i , head, depth = 0)
            else:
                child = self.Suffix_Node(text[i + head.depth + head.len] ,i + head.depth + head.len ,
                                len(text) - i - head.depth - head.len  , head, depth = head.depth + head.len)
            
            head.add_child(child)
            

            
        
            
    def slow_find(self, node, text, suffix_start):
        
        child = node.get_child_by_first_letter(text[suffix_start])
                
        if not child:
            return node
        

        for i in range(1, child.len):
            if text[child.start + i] != text[suffix_start + i]:
                return self.divide_node(child, i, text)
            

        return self.slow_find(child, text, suffix_start + child.len)
    
    def divide_node(self,node, split, text):
        
        
        new_node = self.Suffix_Node(node.letter, node.start, split, node.parent, depth = node.depth)
        #print(self.text[new_node.start:new_node.start+new_node.len])

        node.parent.delete_child(node)
        new_node.parent.add_child(new_node)
        
        
        node.letter = text[node.start+split]
        node.start = node.start+split
        node.len = node.len-split
        node.parent = new_node
        node.depth = new_node.depth + new_node.len
        new_node.add_child(node)
        
        #print(self.text[node.start:node.start+node.len])
        #print()
        
        # print("Dzieci split " + str(new_node.letter) + " " + str(new_node.children))
        
        return new_node
        
             
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def is_in_text(self, text):
        
        pos = 0
        child = self.root
        
        if len(text) == 0:
            return True
        

        while 1:
            
            child = child.get_child_by_first_letter(text[pos])

            
            if child is None:
                return False
            
            if pos+1 == len(text):
                return True

            for i in range(1,child.len):
                
                if self.text[child.start+i] != text[pos + i]:
                    return False
                
                if pos + i >= len(text)-1:
                    return True
                
            pos += child.len
            node = child
            
        return False
    
    def print_Tree(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
    
    

Drzewo sufiksowe tworzone metodą bruteforce. Pozbawione fast find znanego z algorytmu Mc_Creighta tworzy kolejne sufiksy poprzez odnalezienie właściwego miejsca za pomocą slow_find a następnie ewentualny podział istniejących etykiet i dodanie liścia.

# McCreight Tree

In [5]:

class McCreight_Suffix_Tree:
    
    class Suffix_Node:
        def __init__(self, letter, start, length, parent, children = None, link = None, depth = 0):
            
            self.letter = letter
            self.start = start
            self.len = length
            self.parent = parent
            self.depth = depth
            
            self.link = None
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def delete_child(self, child):
            self.children.pop(child.letter, None)
            
            
        def is_child(self,text):
            return text in self.children.keys()
        
        def get_child(self, text):
            return self.children[text]
        
        def get_child_by_first_letter(self, letter):
            for key in self.children.keys():
                if key[0] == letter:
                    return self.children[key]
            return None
        
            
    def __init__(self, text):
        
        self.root = self.Suffix_Node(None, 0, 0, None)
        self.root.link = self.root
        self.root.parent = self.root
        
        text = self.check_text(text)
        self.text = text
        
        head = self.root
        cur_len = 0 
        
        for i in range(len(text)):
            
            if cur_len == head.depth and head.is_child(text[i + cur_len]):
                head, cur_len = self.slow_find(text, i, head, cur_len)
            
            if head.depth > cur_len:
                head = self.split_node(text, head, cur_len)
                

            child = self.Suffix_Node(text[i+cur_len], i,length = len(text)-i , depth = len(text)-i, parent = head)
            
            head.add_child(child)
            
            if head.link is None:
                self.fast_find(text, head, cur_len)
            head = head.link
            cur_len = head.depth
        
    
    def slow_find(self, text, cur_start, head, pref_len):

        while pref_len == head.depth and head.is_child(text[cur_start + pref_len]):
            head = head.get_child(text[cur_start + pref_len])
            pref_len += 1
            
            while pref_len < head.depth and text[cur_start + pref_len] == text[head.start + pref_len]:
                pref_len += 1
        return head, pref_len
    
    def split_node(self, text, node, pref_len):
        
        parent = node.parent
        new_node = self.Suffix_Node(text[node.start + parent.depth], node.start,length=pref_len, depth=pref_len, 
                              parent=parent)
        parent.add_child(new_node)
        node.parent = new_node
        node.letter = text[node.start + pref_len]
        new_node.add_child(node)
        
        return new_node
    

    
    def fast_find(self, text, head, pref_len):
        next_head = head.parent.link
        
        while next_head.depth < pref_len - 1:
            next_head = next_head.get_child(text[head.start + next_head.depth + 1])
        if next_head.depth > pref_len - 1:
            next_head = self.split_node(text, next_head, pref_len - 1)
        head.link = next_head
        
             
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def is_in_text(self, text):
        
        node = self.root
        
        for pos, letter in enumerate(text):
            if pos == node.depth:
                child = node.get_child_by_first_letter(letter)
                if child is None:
                    return False
                node = child
            else:
                if letter != self.text[node.start + pos]:
                    return False
        return True
    
    def print_Tree(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
    
    

Drzewo tworzone za pomocą opisanego na wykładzie algorytmu Mc_Creighta. Do przyspieszenia tworzenia swojej struktury wykorzystuje mechanizmy linków sufiksowych - fast_find.

# Testy poprawności i czasowe

In [29]:


import random
import time

T1 = 'bbb$'
T2 = 'aabbabd'
T3 = 'ababcd'
T4 = 'abcbccd'
max_test_num = 10
T5 = 'g'*6 + 'aafg' + 'h'*2 + 'n'*6 + 'g'*2 + 'c'

with open('1997_714.txt',errors = 'ignore') as f:
    T6 = f.read()

def test(algorithm, sets):

    for num, s in enumerate([T1, T2, T3, T4, T5, T6]):
        if num < sets:
            print('Build for test' + str(num))
            print('Time for 100 builds:')

            start = time.time()
            for i in range(1000):
                tree = algorithm(s)
            end = time.time()

            print(str(round(((end-start)/10),3)) + "ms")
            tree = algorithm(s)

            start = time.time()
            for i in range(0, 10000):
                r = random.randint(0, len(s)-1)
                t = s[r:]

                if not tree.is_in_text(t):
                    return False
            end = time.time()
            print('Search time for 10000 random suffix searches: ' + str(round((end-start),3)) + "ms" )


            print(f'Test num {num+1} passed')
    return True



In [30]:
if test(Trie, 5):
    print('Tests passed') 
else:
    print('Tests failed')



Build for test0
Time for 100 builds:
0.003ms
Search time for 10000 random suffix searches: 0.032ms
Test num 1 passed
Build for test1
Time for 100 builds:
0.005ms
Search time for 10000 random suffix searches: 0.032ms
Test num 2 passed
Build for test2
Time for 100 builds:
0.004ms
Search time for 10000 random suffix searches: 0.032ms
Test num 3 passed
Build for test3
Time for 100 builds:
0.005ms
Search time for 10000 random suffix searches: 0.028ms
Test num 4 passed
Build for test4
Time for 100 builds:
0.03ms
Search time for 10000 random suffix searches: 0.068ms
Test num 5 passed
Tests passed


In [31]:
if test(Link_Trie, 5): # test(Trie, 6) U can do this but this takes eternity to build and a bit MORE then all your RAM  
    print('Tests passed') 
else:
    print('Tests failed')


Build for test0
Time for 100 builds:
0.005ms
Search time for 10000 random suffix searches: 0.032ms
Test num 1 passed
Build for test1
Time for 100 builds:
0.006ms
Search time for 10000 random suffix searches: 0.032ms
Test num 2 passed
Build for test2
Time for 100 builds:
0.004ms
Search time for 10000 random suffix searches: 0.028ms
Test num 3 passed
Build for test3
Time for 100 builds:
0.006ms
Search time for 10000 random suffix searches: 0.032ms
Test num 4 passed
Build for test4
Time for 100 builds:
0.034ms
Search time for 10000 random suffix searches: 0.068ms
Test num 5 passed
Tests passed


In [32]:
if test(Suffix_Tree, 5): # test(Trie, 6) U can do this but this takes eternity to build and a bit MORE then all your RAM  
    print('Tests passed') 
else:
    print('Tests failed')


Build for test0
Time for 100 builds:
0.003ms
Search time for 10000 random suffix searches: 0.04ms
Test num 1 passed
Build for test1
Time for 100 builds:
0.003ms
Search time for 10000 random suffix searches: 0.032ms
Test num 2 passed
Build for test2
Time for 100 builds:
0.002ms
Search time for 10000 random suffix searches: 0.032ms
Test num 3 passed
Build for test3
Time for 100 builds:
0.002ms
Search time for 10000 random suffix searches: 0.032ms
Test num 4 passed
Build for test4
Time for 100 builds:
0.009ms
Search time for 10000 random suffix searches: 0.06ms
Test num 5 passed
Tests passed


In [33]:
if test(McCreight_Suffix_Tree, 5): # test(Trie, 6) U can do this but this takes eternity to build and a bit MORE then all your RAM  
    print('Tests passed') 
else:
    print('Tests failed')

Build for test0
Time for 100 builds:
0.004ms
Search time for 10000 random suffix searches: 0.032ms
Test num 1 passed
Build for test1
Time for 100 builds:
0.004ms
Search time for 10000 random suffix searches: 0.028ms
Test num 2 passed
Build for test2
Time for 100 builds:
0.002ms
Search time for 10000 random suffix searches: 0.028ms
Test num 3 passed
Build for test3
Time for 100 builds:
0.002ms
Search time for 10000 random suffix searches: 0.04ms
Test num 4 passed
Build for test4
Time for 100 builds:
0.008ms
Search time for 10000 random suffix searches: 0.04ms
Test num 5 passed
Tests passed


In [41]:
print('Build for test6')

start = time.time()
tree = Suffix_Tree(T6)
end = time.time()
print(str(round(((end-start)),3)) + "ms")


start = time.time()
for i in range(0, 100):
    r = random.randint(0, len(T6)-1)
    t = T6[r:]

    tree.is_in_text(t)

end = time.time()
print('Search time for 100 random suffix searches: ' + str(round((end-start),3)) + "ms" )

print('Test num 6 passed')

Build for test6
6.073ms
Search time for 100 random suffix searches: 4.508ms
Test num 6 passed


In [42]:
print('Build for test6')

start = time.time()
tree = McCreight_Suffix_Tree(T6)
end = time.time()
print(str(round(((end-start)),3)) + "ms")


start = time.time()
for i in range(0, 100):
    r = random.randint(0, len(T6)-1)
    t = T6[r:]

    tree.is_in_text(t)

end = time.time()
print('Search time for 100 random suffix searches: ' + str(round((end-start),3)) + "ms" )

print('Test num 6 passed')

Build for test6
4.209ms
Search time for 100 random suffix searches: 2.192ms
Test num 6 passed


# Podsumowanie

Powyższe implementacje drzew działają poprawnie. Pozwalają na szybkie sprawdzenie czy zadana fraza jest sufiksem danego tekstu, dużo szybciej niż pozwalają na to algorytmy wyszukiwania wzorca.

Czas wyszukiwania sufiksu w strukturach jest podobny dla wszystkich drzew. Spowodowane jest to tą samą metodą przeszukiwania struktur, przechodzeniu od węzła do węzła.

Najgorszy teoretyczny i praktyczny czas budowania zapewniają drzewa Trie. Powstają wolno oraz mają duże zapotrzebowanie pamięciowe. Złożoność budowy Trie wynosi O(n^2)

Czas budowania Suffix_Tree jest dużo lepszy niż czas budowania struktur Trie. Pomimo podejścia brute_force Suffix_Tree jest znaczną oszczędnością pamięci i czasu, pozwala na obsługę dużych objętościowo tekstów bez znacznych strat czasowych.

Najszybszą struktur jest drzewo sufiksowe oparte na algorytmie Mc_Creighta. Poprawia już i tak dobre wyniki drzewa sufiksowego nawet dwukrotnie.